In [57]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import re
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

df_test = pd.read_csv("../ventas_limpias.csv")

print(df_test)
# [1.048.574  rows x 11 columns]
# 11 Columnas
# 1.048.575 rows

                   ciudad       fecha producto      tipo_producto  cantidad  \
0                santiago  2025-10-30    arepa          abarrotes       2.0   
1                 cordoba  2025-11-17    arepa          abarrotes       7.0   
2            barranquilla  2025-10-22    leche             lacteo       9.0   
3                new york  2025-10-20   cereal             lacteo       3.0   
4                  madrid  2025-10-20    leche              hogar       2.0   
...                   ...         ...      ...                ...       ...   
1048570       guadalajara  2025-10-23   yogurt             lacteo       9.0   
1048571  ciudad de mexico  2025-11-13  gaseosa              hogar       7.0   
1048572              lima  2025-10-30    arepa             bebida       8.0   
1048573            madrid  2025-10-23     cafe          abarrotes      10.0   
1048574             cusco  2025-10-22    queso  alimentopercedero       1.0   

         precio_unitario tipo_de_venta tipo_cliente

In [116]:
#Carga el archivo .env
load_dotenv(override=True)

#Variables de entorno

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")


URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(URL)

conn = engine.connect()
try:
    print("Connection Succesfull!!" if conn else "")

except Exception as e:
    print("Error al conectar la base datos en", e)
    
set = conn.execute(text("SET search_path TO riwi_ventas"))


Connection Succesfull!!


### Functions

In [121]:
#Carga el archivo .env
load_dotenv(override=True)

#Variables de entorno

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")


URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(URL)

conn = engine.connect()
try:
    print("Connection Succesfull!!" if conn else "")

except Exception as e:
    print("Error al conectar la base datos en", e)
    
set = conn.execute(text("SET search_path TO riwi_ventas"))

table_list = pd.read_sql(text("SELECT table_name FROM information_schema.tables WHERE table_schema = 'riwi_ventas';"),conn)

# def cleaner(table):
#     dinamic_df = pd.read_sql(text(f"select * from riwi_ventas.{table}"),conn)

#     dinamic_df = dinamic_df[dinamic_df.isnull().mean(axis=1) <= 0.50]
#     a = dinamic_df.columns
#     a.tolist()

#     false_count = 0
#     for c in a:
#         if c ==False:
#             false_count +=1
#     print("Hay: ",false_count, " datos nulos")
#     return dinamic_df

def cleaner(table, conn):
    original_df = pd.read_sql(text(f'SELECT * FROM riwi_ventas."{table}"'), conn)

    clean_df = original_df[original_df.isnull().mean(axis=1) <= 0.50]

    print(f"[{table}] Filas originales: {len(original_df)}, Filas limpias: {len(clean_df)}")

    return original_df, clean_df

# for i in range(len(table_list)):
#     df = cleaner(table_list["table_name"][i],conn)
#     print(f"{table_list['table_name'][i]}: {df.columns.tolist()}\n")

def get_primary_key(table):
    query = text(f"""
        SELECT a.attname AS pk
        FROM pg_index i
        JOIN pg_attribute a 
            ON a.attrelid = i.indrelid 
           AND a.attnum = ANY(i.indkey)
        WHERE i.indrelid = 'riwi_ventas."{table}"'::regclass
          AND i.indisprimary;
    """)
    
    result = conn.execute(query).fetchone()

    return result[0] if result else None

def delete_rows(table, pk, ids):
    if not ids:
        print("No hay IDs para eliminar.")
        return

    query = text(f"""
        DELETE FROM riwi_ventas."{table}"
        WHERE {pk} = ANY(:ids)
    """)

    conn.execute(query, {"ids": list(ids)})
    print(f"✔ Eliminadas {len(ids)} filas.")

def insert_clean_rows(table, df):
    if df.empty:
        print("No hay filas limpias para insertar.")
        return

    df = df.where(pd.notnull(df), None)

    cols = df.columns.tolist()
    colnames = ", ".join([f'"{c}"' for c in cols])
    placeholders = ", ".join([f":{c}" for c in cols])

    query = text(f"""
        INSERT INTO riwi_ventas."{table}" ({colnames})
        VALUES ({placeholders})
    """)

    for _, row in df.iterrows():
        conn.execute(query, row.to_dict())

    print(f"✔ Insertadas {len(df)} filas limpias.")



with engine.begin() as conn:

    for table in table_list["table_name"]:

        print(f"\n=== Procesando tabla: {table} ===")

        original_df, clean_df = cleaner(table, conn)

        # obtener primary key
        pk = get_primary_key(table)

        if pk is None:
            print(f"No PK en tabla {table}, saltando…")
            continue

        # identificar IDs eliminados
        deleted_ids = set(original_df[pk]) - set(clean_df[pk])

        # borrar filas sucias
        delete_rows(table, pk, deleted_ids)

        # reinsertar las filas limpias
        insert_clean_rows(table, clean_df)

        print(f"✔ Tabla {table} actualizada correctamente.")



# df = pd.read_sql(text("SELECT * FROM factura_ventas;"), conn)
# #dfc = pd.read_sql(text("select * from ciudad;"),conn)

# cols = df.columns.to_list()

# # a = dfc["nombre_ciudad"].notnull()
# # a.tolist()

# false_count = 0
# for c in a:
#     if c ==False:
#         false_count +=1
# print("Hay: ",false_count, " datos nulos")




Connection Succesfull!!

=== Procesando tabla: tipo_producto ===
[tipo_producto] Filas originales: 7, Filas limpias: 7


TypeError: 'CursorResult' object is not callable

In [101]:
table_list = pd.read_sql(text("SELECT table_name FROM information_schema.tables WHERE table_schema = 'riwi_ventas';"),conn)

current_schema = pd.read_sql("SHOW search_path;", conn)
print(current_schema)

def cleaner(table):
    dinamic_df = pd.read_sql(text(f"SELECT * FROM {table}"), conn)

    # limpiar
    clean_df = dinamic_df[dinamic_df.isnull().mean(axis=1) <= 0.50]

    # detectar nulos
    deleted_rows = len(dinamic_df) - len(clean_df)
    print(f"[{table}] Filas eliminadas por nulos: {deleted_rows}")

    return dinamic_df, clean_df


with engine.begin() as conn:

    for table in table_list["table_name"]:

        print(f"\n=== Procesando tabla: {table} ===")

        original_df, clean_df = cleaner(table)

        # obtener primary key
        pk = get_primary_key(table)

        if pk is None:
            print(f"No PK en tabla {table}, saltando…")
            continue

        # identificar IDs eliminados
        deleted_ids = set(original_df[pk]) - set(clean_df[pk])

        # borrar filas sucias
        delete_rows(table, pk, deleted_ids)

        # reinsertar las filas limpias
        insert_clean_rows(table, clean_df)

        print(f"✔ Tabla {table} actualizada correctamente.")


ResourceClosedError: This Connection is closed